# Sequence를 처리하기 위한 RNN 

1. 주어진 데이터를 RNN에 넣을 수 있는 형태로 만듭니다.
2. 기본적인 RNN 사용법 및 적용법을 익힙니다.
3. LSTM, GRU의 사용법 및 적용법을 익힙니다.

In [1]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch

## 데이터 전처리

아래의 sample data를 확인해봅시다.  
전체 단어 수와 pad token의 id도 아래와 같습니다.

In [2]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [3]:
data[0]

[85, 14, 80, 34, 99, 20, 31, 65, 53, 86, 3, 58, 30, 4, 11, 6, 50, 71, 74, 13]

Padding 처리를 해주면서 padding 전 길이도 저장합니다.

In [4]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 12365.28it/s]


In [5]:
print(data)
print(valid_lens)

[[85, 14, 80, 34, 99, 20, 31, 65, 53, 86, 3, 58, 30, 4, 11, 6, 50, 71, 74, 13], [62, 76, 79, 66, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [93, 77, 16, 67, 46, 74, 24, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [19, 83, 88, 22, 57, 40, 75, 82, 4, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [70, 28, 30, 24, 76, 84, 92, 76, 77, 51, 7, 20, 82, 94, 57, 0, 0, 0, 0, 0], [58, 13, 40, 61, 88, 18, 92, 89, 8, 14, 61, 67, 49, 59, 45, 12, 47, 5, 0, 0], [22, 5, 21, 84, 39, 6, 9, 84, 36, 59, 32, 30, 69, 70, 82, 56, 1, 0, 0, 0], [94, 21, 79, 24, 3, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [80, 80, 33, 63, 34, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [87, 32, 79, 65, 2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80, 0, 0]]
[20, 5, 8, 10, 15, 18, 17, 6, 6, 18]


In [6]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

In [7]:
batch.shape

torch.Size([10, 20])

In [8]:
batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

In [9]:
print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

## RNN 사용해보기

RNN에 넣기 전 word embedding을 위한 embedding layer를 만듭니다.

In [10]:
batch.shape

torch.Size([10, 20])

In [11]:
embedding_size = 256

## TODO ##
embedding = nn.Embedding(vocab_size, embedding_size)

# d_w: embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

In [12]:
batch.shape # [batch, token]

torch.Size([10, 20])

In [13]:
batch_emb.shape # [batch(문장의 개수), token(문장의 길이), vectar(벡터의 차원)]

torch.Size([10, 20, 256])

RNN 모델 및 초기 hidden state를 정의

- batch_emb 변수를 RNN에 넣을 예정입니다.
- torch 공식 문서를 참조하여, RNN 모델을 정의해보세요. 
- input size는 어떻게 되어야 하나요?

In [14]:
N, L, H_in = batch_emb.shape
N, L, H_in

(10, 20, 256)

In [15]:
hidden_size = 512  # RNN의 hidden size
num_layers = 1  # 쌓을 RNN layer의 개수
num_dirs = 1  # 1: 단방향 RNN, 2: 양방향 RNN

rnn = nn.RNN(
    # TODO #
    input_size = embedding_size,
    hidden_size = hidden_size,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    batch_first = False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

### **Vanilla RNN 활용법**

RNN에 batch data를 넣으면 아래와 같이 2가지 output을 얻습니다.


*   `hidden_states`: 각 time step에 해당하는 hidden state들의 묶음.
*   `h_n`: 모든 sequence를 거치고 나온 마지막 hidden state.

torch의 RNN 문서를 참조하여서, ``batch_emb``변수를 rnn에 input으로 넣어보세요.
나온 결과의 shape도 출력해보세요. 

In [ ]:

# hidden_states: hidden layer
# h_n: final hidden layer

In [16]:
hidden_states, h_n = rnn(batch_emb.transpose(0, 1), h_0) 

print(hidden_states.shape)
print(h_n.shape) 

torch.Size([20, 10, 512])
torch.Size([1, 10, 512])


In [17]:
hidden_states[-1]

tensor([[ 0.3812,  0.5036,  0.6377,  ..., -0.6816, -0.4591,  0.0435],
        [ 0.6007,  0.2902, -0.4161,  ...,  0.6932, -0.4274,  0.7509],
        [ 0.4678,  0.3690, -0.3554,  ...,  0.7706, -0.3879,  0.6034],
        ...,
        [ 0.6659,  0.3952, -0.2633,  ...,  0.7604, -0.4012,  0.5766],
        [ 0.6659,  0.3952, -0.2632,  ...,  0.7604, -0.4013,  0.5765],
        [ 0.6659,  0.3952, -0.2633,  ...,  0.7604, -0.4012,  0.5765]],
       grad_fn=<SelectBackward0>)

In [18]:
h_n

tensor([[[ 0.3812,  0.5036,  0.6377,  ..., -0.6816, -0.4591,  0.0435],
         [ 0.6007,  0.2902, -0.4161,  ...,  0.6932, -0.4274,  0.7509],
         [ 0.4678,  0.3690, -0.3554,  ...,  0.7706, -0.3879,  0.6034],
         ...,
         [ 0.6659,  0.3952, -0.2633,  ...,  0.7604, -0.4012,  0.5766],
         [ 0.6659,  0.3952, -0.2632,  ...,  0.7604, -0.4013,  0.5765],
         [ 0.6659,  0.3952, -0.2633,  ...,  0.7604, -0.4012,  0.5765]]],
       grad_fn=<StackBackward0>)

In [19]:
hidden_states[:, -1, :], h_n.reshape(10, 1, 512)

(tensor([[ 0.5920,  0.3999, -0.3571,  ...,  0.2564, -0.4519, -0.7771],
         [ 0.0773, -0.1084,  0.6425,  ..., -0.2324, -0.2264,  0.1333],
         [-0.0178,  0.5262, -0.0154,  ..., -0.1138,  0.4017, -0.6461],
         ...,
         [ 0.6660,  0.3953, -0.2632,  ...,  0.7604, -0.4012,  0.5765],
         [ 0.6660,  0.3952, -0.2633,  ...,  0.7604, -0.4013,  0.5766],
         [ 0.6659,  0.3952, -0.2633,  ...,  0.7604, -0.4012,  0.5765]],
        grad_fn=<SliceBackward0>),
 tensor([[[ 0.3812,  0.5036,  0.6377,  ..., -0.6816, -0.4591,  0.0435]],
 
         [[ 0.6007,  0.2902, -0.4161,  ...,  0.6932, -0.4274,  0.7509]],
 
         [[ 0.4678,  0.3690, -0.3554,  ...,  0.7706, -0.3879,  0.6034]],
 
         ...,
 
         [[ 0.6659,  0.3952, -0.2633,  ...,  0.7604, -0.4012,  0.5766]],
 
         [[ 0.6659,  0.3952, -0.2632,  ...,  0.7604, -0.4013,  0.5765]],
 
         [[ 0.6659,  0.3952, -0.2633,  ...,  0.7604, -0.4012,  0.5765]]],
        grad_fn=<ReshapeAliasBackward0>))

In [20]:
hidden_states.shape, h_n.shape

(torch.Size([20, 10, 512]), torch.Size([1, 10, 512]))

마지막 hidden state를 이용하여 text classification task에 적용할 수 있습니다.

In [21]:
num_classes = 2
classification_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = classification_layer(h_n.squeeze(0))  # (1, B, d_h) => (B, C)
print(output.shape)

torch.Size([10, 2])


각 time step에 대한 hidden state를 이용하여 token-level의 task를 수행할 수도 있습니다.

In [22]:
num_classes = 5
entity_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = entity_layer(hidden_states)  # (L, B, d_h) => (L, B, C)
print(output.shape)

torch.Size([20, 10, 5])


### **LSTM 활용법**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

- batch_emb 변수를 LSTM에 넣을 예정입니다.
- torch 공식 문서를 참조하여, LSTM 모델을 정의해보세요. 
- input size는 어떻게 되어야 하나요?

In [36]:
hidden_size = 512
num_layers = 1
num_dirs = 1

## TODO ##
lstm = nn.LSTM(
    input_size = embedding_size,
    hidden_size = hidden_size,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    batch_first = True
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

torch의 LSTM 문서를 참조하여서, ``batch_emb``변수를 rnn에 input으로 넣어보세요.
나온 결과의 shape도 출력해보세요. 

In [37]:
## TODO ##
output, (h_n, c_n) = lstm(batch_emb)

In [38]:
output.shape, h_n.shape, c_n.shape

(torch.Size([10, 20, 512]), torch.Size([1, 10, 512]), torch.Size([1, 10, 512]))

In [ ]:
# seq 2 seq

#                    안 녕
# h1 h2 h3 h4 h5  c5 -> (lstm)

# h e l l o

In [39]:
output[:, -1, :]

tensor([[ 0.0498, -0.0304, -0.1301,  ..., -0.1076,  0.0742, -0.0442],
        [-0.0247, -0.2109, -0.1264,  ..., -0.2118, -0.1255,  0.0517],
        [-0.0324, -0.2428, -0.0989,  ..., -0.2317, -0.0977,  0.0966],
        ...,
        [ 0.0248, -0.3107, -0.1333,  ..., -0.3804, -0.1434,  0.1037],
        [ 0.0238, -0.3108, -0.1335,  ..., -0.3790, -0.1432,  0.1042],
        [ 0.0240, -0.3110, -0.1331,  ..., -0.3797, -0.1431,  0.1037]],
       grad_fn=<SliceBackward0>)

### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.

In [ ]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [ ]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [ ]:
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

In [ ]:
for t in range(max_len):
  input_emb = embedding(input_id).unsqueeze(0)  # (1, B, d_w)
  output, hidden = gru(input_emb, hidden)  # output: (1, B, d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output)  # (1, B, V)
  probs, top_id = torch.max(output, dim=-1)  # probs: (1, B), top_id: (1, B)

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.


In [ ]:
num_layers = 2
num_dirs = 2
dropout=0.1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=True if num_dirs > 1 else False
)

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [ ]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

In [ ]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [ ]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)